## Установка pandas

In [1]:
### !pip3 install pandas

In [1]:
import pandas as pd

## Чтение данных и фильтрация таблиц

In [18]:
user = pd.read_csv('user.csv')

In [19]:
user.birthday = pd.to_datetime(user.birthday, format='%d.%m.%Y')
user.last_visited = pd.to_datetime(user.last_visited, format='%d.%m.%Y %H:%M:%S')

In [20]:
user.head()

,id,gender,birthday,grade,region,last_visited
0,1,М,2007-11-02,9.0,Город Санкт-Петербург,2023-08-23 03:04:14
1,2,Ж,2009-01-29,8.0,Город Москва,2023-09-07 09:21:35
2,3,М,2007-05-09,NaN,Город Санкт-Петербург,2023-08-23 18:32:18
3,4,Ж,2007-09-27,9.0,Город Москва,2023-08-18 07:34:11
4,5,Ж,2007-08-04,NaN,Город Москва,2023-09-02 18:31:30


In [21]:
user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            1000 non-null   int64         
 1   gender        1000 non-null   object        
 2   birthday      1000 non-null   datetime64[ns]
 3   grade         831 non-null    float64       
 4   region        1000 non-null   object        
 5   last_visited  1000 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(1), object(2)
memory usage: 47.0+ KB


### Задание 1

Сколько в таблице `user` представлено учеников, родившихся в сентябре?

In [22]:
(user.birthday.dt.month == 9).sum()

73

### Задание 2

Какое количество учеников в последний раз посещали платформу в первые $14$ дней сентября?

In [24]:
((user.last_visited.dt.month == 9) & (user.last_visited.dt.day.between(1, 14))).sum()

377

### Задание 3

На сколько в данных больше девятиклассников, чем девятиклассниц?

In [41]:
((user.gender == 'М') & (user.grade == 9)).sum() - ((user.gender == 'Ж') & (user.grade == 9)).sum()

5

### Задание 4

Какой процент учеников, у которых указан класс, обучается в $9$-м классе?

Ответ округлите до целых.

In [45]:
((user.grade == 9).sum() / user.grade.count() * 100).round()

24.0

### Задание 5

Сколько в данных представлено учеников из регионов, в названиях которых в качестве подстроки содержится слово «город»?

При расчёте статистики приведите названия регионов к строчному написанию.

In [72]:
user.region.str.lower().str.contains('город').sum()
# user[(user.region.str.contains('город')) & (user.gender == 'М')].shape

765

## Группировка, агрегация и соединение таблиц

In [2]:
course = pd.read_csv('course.csv')

In [3]:
user_course_progress = pd.read_csv('user_course_progress.csv')

**Относительным прогрессом** будем называть отношение суммарного числа баллов, которые ученик набрал за курс (колонка `progress` в таблице `user_course_progress`), к максимальному количеству баллов, которые ученик в принципе мог набрать за данный курс (колонка `max_progress` в таблице `course`).

С помощью представленных данных найдите ответ на следующий вопрос: в курсах какого предмета больше всего учеников, чей относительный прогресс не меньше $0.8$?

В рамках следующих заданий вам будет предложено, действуя поэтапно, найти ответ на этот вопрос.

In [5]:
course.head()

,id,subject,date_started,max_score
0,1,Химия,2023-04-04,108.0
1,2,Физика,2023-01-31,137.0
2,3,Биология,2023-09-14,NaN
3,4,Химия,2023-03-31,118.0
4,5,Физика,2023-11-30,NaN


In [7]:
user_course_progress.head()

,user_id,course_id,progress
0,392,1,31.68
1,217,1,97.53
2,843,1,81.50
3,417,1,15.92
4,605,1,103.31


### Задание 1

Произведите соединение таблиц `course` и `user_course_progress`.

В качестве ответа укажите количество строчек итоговой таблицы, в которых представлены ученики курсов по химии.

In [25]:
df = pd.merge(course, user_course_progress, left_on='id', right_on='course_id')
df.head()

,id,subject,date_started,max_score,user_id,course_id,progress
0,1,Химия,2023-04-04,108.0,392,1,31.68
1,1,Химия,2023-04-04,108.0,217,1,97.53
2,1,Химия,2023-04-04,108.0,843,1,81.50
3,1,Химия,2023-04-04,108.0,417,1,15.92
4,1,Химия,2023-04-04,108.0,605,1,103.31


In [18]:
df[df.subject=='Химия'].subject.count()

678

### Задание 2

Обратите внимание, что у части курсов нет значения в колонке `max_score`. Уберите из полученной в предыдщуем задании таблицы все строчки, в которых пропущено значение колонки `max_score`.

В качестве ответа укажите количество строчек в итоговой таблице.

In [26]:
df.shape

(1711, 7)

In [27]:
df = df.dropna(subset=['max_score'])

In [29]:
df.shape[0]

1031

### Задание 3

Добавьте в полученную в предыдущем задании таблицу колонку `relative_progress`, в которой для каждого ученика будет храниться значение его относительного прогресса в конкретном курсе.

В качестве ответа укажите среднее значение относительного прогресса в таблице. Ответ округлите до двух знаков после запятой с помощью функции `round`, которая встроена в Series.

In [33]:
df['relative_progress'] = (df.progress / df.max_score)

In [38]:
round(df.relative_progress.mean(), 2)

0.35

### Задание 4

Оставьте в полученной в предыдущем задании таблице только те строчки, в которых значение относительного прогресса не меньше $0.8$.

В качестве ответа укажите число строчек в получившейся таблице.

In [39]:
df_best = df[df.relative_progress>=0.8]

In [41]:
df_best.shape[0]

159

### Задание 5

Произведите группировку строчек таблицы по предметам и посчитайте, сколько учеников с относительным прогрессом не меньше $0.8$ оказалось в каждом из предметов.

В качестве ответа заполните таблицу, в которой каждому предмету сопоставлено число таких учеников.

Если один и тот же ученик учился в разных курсах одного и того же предмета и в каждом из этих курсов достиг относительного прогресса не менее $0.8$, при подсчёте учтите его несколько раз.

In [45]:
df_best.subject.value_counts()

Химия         105
Математика     30
Физика         24
Name: subject, dtype: int64

### Задание 6

В курсах какого предмета больше всего учеников, достигших относительного прогресса не менее $0.8$?

In [46]:
df_best.subject.value_counts().idxmax()

'Химия'

In [30]:
df.head(1)

,id,subject,date_started,max_score,user_id,course_id,progress
0,1,Химия,2023-04-04,108.0,392,1,31.68
